# Basic SQLAlchemy ORM manipulations

In [ ]:
import json
from os import environ
import pyodbc
import urllib
import pandas as pd

from sqlalchemy import create_engine, text, func
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker, aliased, selectinload, joinedload
from sqlalchemy import Column, ForeignKey, Integer, String, Float
from sqlalchemy.orm import relationship, backref
from sqlalchemy.orm.exc import MultipleResultsFound
from sqlalchemy.sql import exists

# Setup

In [ ]:
driver = environ.get('SQL_DRIVER', '{ODBC Driver 17 for SQL Server}')
host = environ.get('SQL_HOST', 'sql-fabulous')
db = environ.get('SQL_DB', 'ScratchDB')
user = environ.get('SQL_USER', 'sa')
pw = environ.get('SQL_PASSWORD', 'HelloWorld1')

con_str = f'DRIVER={driver};SERVER={host};DATABASE={db};UID={user};PWD={pw}'

params = urllib.parse.quote_plus(con_str)  

# 'echo' emits generated sql
engine = create_engine(f"mssql+pyodbc:///?odbc_connect={params}", echo=True)

# Define the catalog of classes

In [ ]:
Base = declarative_base()


class Survey(Base):
    __tablename__ = 'DirectionalSurvey'
    
    ID = Column(Integer, autoincrement=False, primary_key=True)
    API = Column(String(32), nullable=True)
    WKID = Column(String(32), nullable=True)
    FIPS = Column(String(4), nullable=True)
    STATUS_CODE = Column(String(1), nullable=False)

    def __repr__(self):
        return f"Survey(ID={self.ID}, API={self.API}, STATUS={self.STATUS_CODE})"

class SurveyReport(Base):
    __tablename__ = 'SurveyReport'
    
    ID = Column(Integer, autoincrement=False, primary_key=True)
    DirectionalSurveyId = Column(Integer, ForeignKey('DirectionalSurvey.ID'), nullable=False)
    Azimuth = Column(Float, nullable=True)
    MD = Column(Float, nullable=True)
    Inclination = Column(Float, nullable=True)
    STATUS_CODE = Column(String(1), nullable=False)

    # Relationship
    survey = relationship(Survey, backref=backref('stations', uselist=True))
    #survey = relationship(Survey, back_populates='stations')

    def __repr__(self):
        return f"Report(ID={self.ID}, FK={self.DirectionalSurveyId}, STATUS={self.STATUS_CODE})"

# Inspect underlying metadata

In [ ]:
SurveyReport.__table__

# Create Schema

In [ ]:
Base.metadata.create_all(engine, checkfirst=True)

# Create Bean

In [ ]:
# Automatically creates a constructor accepting kwargs
survey4 = Survey(ID=4, API='API4', WKID='0004', STATUS_CODE='C')
survey4

# Create session - interface to the DB

In [ ]:
Session = sessionmaker(bind=engine)
session = Session()

# Insert

In [ ]:
session.add(survey4)
#session.rollback()
session.commit()

# Update

In [ ]:
survey_tmp = session.query(Survey).filter_by(API = 'API4').first()
survey_tmp.WKID = 'WKID4'
survey_tmp.FIPS = '0004'

session.add_all([survey_tmp])
session.commit()

# Querying

In [ ]:
for i in session.query(Survey).order_by(Survey.ID):
    print(f"{i.ID}, {i.API}")

In [ ]:
for id, api in session.query(Survey.ID, Survey.API):
    print(f"{id}, {api}")

In [ ]:
for row in session.query(Survey, Survey.ID).all():
    print(f"{row.Survey}, {row.Survey.stations}, {row.ID}")

In [ ]:
for row in session.query(Survey, Survey.ID.label('SurveyID')).all(): # field aliasing
    print(f"{row.Survey}, {row.SurveyID}")

In [ ]:
# table aliasing
DirectionalSurvey = aliased(Survey, name='DirectionalSurvey')
for row in session.query(DirectionalSurvey, DirectionalSurvey.ID):
    print(f"{row.DirectionalSurvey}, {row.ID}")

In [ ]:
# Can use python slicer to limit results.
for row in session.query(Survey).order_by(Survey.ID)[-1::-1]:
    print(f"{row.ID}")

In [ ]:
# Where clause
for s in session.query(Survey).filter(Survey.API == "API3"):
    print(f"{s}")

In [ ]:
# Where clause AND
# ilike applies a lower to both sides
for s in session.query(Survey)\
            .filter(Survey.API.startswith("API"))\
            .filter(Survey.WKID.like("WK%"))\
            .filter(Survey.WKID.ilike("WK%"))\
            .filter(Survey.STATUS_CODE.in_("N")):
    print(f"{s}")

In [ ]:
# IN/NOT IN Sub-query
for s in session.query(Survey)\
            .filter(~Survey.ID.in_(
                session.query(SurveyReport.DirectionalSurveyId)
            )):
    print(f"{s}")

In [ ]:
for sr in session.query(SurveyReport):
    print(f"{sr}")

# Other query operators

In [ ]:
# "==" is the same as "is_"
# "!=" is the same as "isnot"
# Compound filters are the same as "and_"
# "or_" can also be used
# ""== None" is the same as "IS / IS NOT NULL"
# contains()

# Result Operators all, first, one

In [ ]:
for r in session.query(Survey).all(): # returns list
    print(r)

In [ ]:
r = session.query(Survey).first() # returns single tuple
print(r)

In [ ]:
try:
    # Fails if more than one exists 
    # "one_or_none()" returns None instead of failing.
    r = session.query(Survey).one() 
except MultipleResultsFound as e:
    print(e)

# Bind Parameters

In [ ]:
for s in session.query(Survey).filter(text("id<:rec and API=:api")).\
                            params(rec=4, api='API2'):
    print(s)

# Literal SQL

In [ ]:
s = session.query(Survey).from_statement(
    text("SELECT ID, API FROM DirectionalSurvey where API=:api")).\
        params(api='API1').first()

s

# COUNT()

In [ ]:
g = session.query(func.count(Survey.STATUS_CODE), Survey.STATUS_CODE)\
        .group_by(Survey.STATUS_CODE).all()
g

In [ ]:
session.query(func.count(Survey.ID)).scalar() # count(*)

# Navigating Relations

In [ ]:
r = session.query(Survey).first() # returns single tuple
print(r.stations[0].survey.API)

# Joins

In [ ]:
# Old SQL cross-join style
for s, sr in session.query(Survey, SurveyReport).\
                filter(Survey.ID == SurveyReport.DirectionalSurveyId).\
                filter(Survey.STATUS_CODE.in_(['N'])):
    print(f"{s}, {sr}")

In [ ]:
# Standard style
# join can be explicit: 
# join(SurveyReport, Survey.ID == SurveyReport.DirectionalSurveyId)
# outerjoin() also available
# Use table aliasing to join to the same table more than once.
for row in session.query(Survey).\
                join(SurveyReport).\
                filter(Survey.STATUS_CODE.in_(['N'])):
    print(f"{row}, {row.stations}")

# Sub-queries

In [ ]:
# Build named sub-query
stmt = session.query(SurveyReport.DirectionalSurveyId, func.count('*').\
            label('report_count')).\
            group_by(SurveyReport.DirectionalSurveyId).subquery()

# Join to parent table
for u, count in session.query(Survey, stmt.c.report_count).\
                    outerjoin(stmt, Survey.ID == stmt.c.DirectionalSurveyId).\
                    order_by(Survey.ID):
    print(u, count)

In [ ]:
# Exists/Not Exists use "~"
# Use has() for many to 1 relations
for s in session.query(Survey).\
        filter(Survey.stations.any(SurveyReport.STATUS_CODE.in_(['N', 'C']))): # or any()
    print(s)

# Optimizations

In [ ]:
# Loads all related objects eagerly
s1 = session.query(Survey).\
        options(selectinload(Survey.stations)).\
        filter(Survey.API == 'API1').one()

s1

In [ ]:
# Loads all related objects eagerly
jack = session.query(Survey).\
        options(joinedload(Survey.stations)).\
        filter_by(API='API1').one()
jack

# Deletes

In [ ]:
sr1 = session.query(SurveyReport)\
        .filter(SurveyReport.ID == 6)\
        .one()
sr1

In [ ]:
session.delete(sr1)

In [ ]:
session.commit()